In [ ]:
import pandas as pd
import numpy as np

fund = pd.read_excel(r'四只开放式股票型基金的净值.xlsx',header = 0,index_col = 0)

# 夏普比率
def SR(Rp,Rf,Vp):
    return (Rp - Rf)/Vp
   
R_fund = np.log(fund/fund.shift(1)) #基金的日对数收益率
R_fund = R_fund.dropna()
R_mean = R_fund.mean()*252 #计算全部3年的平均年化收益率
sigma = R_fund.std()*np.sqrt(252) #计算全部3年的平均年化收益波动率
R_f = 0.015 #无风险利率
SR_3years = SR(R_mean,R_f,sigma)

print('2016-2018平均3年的夏普比率：\n',SR_3years)


In [ ]:
#索提诺比率
def SOR(Rp,Rf,Vp):
    return (Rp - Rf)/Vp

Vp_lower = np.zeros_like(R_mean)

for i in range(len(Vp_lower)):
    #生产收益率为负的时间序列
    R_neg = R_fund.iloc[:,i][R_fund.iloc[:,i] < 0] 
    #年化下行收益标准差
    Vp_lower[i] = np.sqrt(252)*np.sqrt(np.sum(R_neg**2)/len(R_neg))
    print(R_fund.columns[i]+'收益下行标准差:',Vp_lower[i])
    
SOR_3years = SOR(R_mean,R_f,Vp_lower)

print('2016-2018平均3年的索提诺比率：\n',SOR_3years)


In [ ]:
import statsmodels.api as sm

HS300 = pd.read_excel(r'C:\Users\Administrator\Desktop\沪深300指数.xlsx',header = 0,index_col = 0)

#计算日收益率
R_HS300 = np.log(HS300/HS300.shift(1))
R_HS300 = R_HS300.dropna()

# 特雷诺比率
def TR(Rp,Rf,beta):
    return (Rp - Rf)/beta

betas = np.zeros_like(R_mean)
cons = np.zeros_like(R_mean)
X = R_HS300
X_addcons = sm.add_constant(X)

for i in range(len(R_mean)):
    #获取基金产品时间序列
    Y = R_fund.iloc[:,i]
    #使用线性回归计算beta值
    model = sm.OLS(endog=Y,exog=X_addcons)
    result = model.fit()
    cons[i] = result.params[0]
    betas[i] = result.params[1]
    print(R_fund.columns[i] + 'beta = ',betas[i])
    
TR_3years = TR(R_mean,R_f,betas)
print('2016-2018平均3年的特雷诺比率：\n',TR_3years)


In [ ]:
# 信息比率
def IR(Rp,Rb,te):
    return (Rp - Rb)/te

TE_fund = np.zeros_like(R_mean)

for i in range(len(R_mean)):
    TD = np.array(R_fund.iloc[:,i]) - np.array(R_HS300.iloc[:,0])
    TE_fund[i] = TD.std()*np.sqrt(252)
    print(R_fund.columns[i] + '跟踪误差',TE_fund[i])
    
R_mean_HS300 = np.array(R_HS300.mean()*252)
IR_3years = IR(R_mean,R_mean_HS300,TE_fund)
print('2016-2018平均3年的信息比率：\n',IR_3years)
